In [1]:
from datetime import datetime
import datetime as dt
import os
from operator import itemgetter
import shutil 
import pickle
import time
import json

from django.contrib.gis.geos.geometry import GEOSGeometry
from osgeo import osr
from osgeo import gdal
from pyproj import Proj
from fiona.crs import from_epsg
from fiona.crs import to_string
from sklearn.cluster import KMeans
import geopandas as gpd
from affine import Affine
import xarray as xr
import numpy as np
from rasterio.features import rasterize
from shapely.geometry import shape
import fiona
import glob
from dask.distributed import Client
from datacube.drivers.netcdf import write_dataset_to_netcdf
from datacube.api import GridWorkflow
import rasterio
from rasterio.mask import mask

from madmex.wrappers import gwf_query
from madmex.util.xarray import to_float, to_int
from madmex.models import PredictObject, TrainClassification
from madmex.models import Region

2020-03-10 16:28:26,276 - datacube.drivers.driver_cache - driver_cache safe_load: Failed to resolve driver datacube.plugins.index::s3aio_index
2020-03-10 16:28:26,279 - datacube.drivers.driver_cache - driver_cache safe_load: Error was: UndefinedEnvironmentName("'extra' does not exist in evaluation environment.",)
/home/madmex_user/.local/lib/python3.6/site-packages/distributed/worker.py:3269: UserWarning: Large object of size 25.02 MB detected in task graph: 
2020-03-10 16:36:40,011 - madmex.wrappers - loggerwriter write: /home/madmex_user/.local/lib/python3.6/site-packages/distributed/worker.py:3269: UserWarning: Large object of size 25.02 MB detected in task graph: 
  (array([[False, False, False, ...,  True,  True,   ... , 3200, None)))
2020-03-10 16:36:40,017 - madmex.wrappers - loggerwriter write:   (array([[False, False, False, ...,  True,  True,   ... , 3200, None)))
Consider scattering large objects ahead of time
2020-03-10 16:36:40,020 - madmex.wrappers - loggerwriter write: C

In [2]:
client = Client(n_workers=5,memory_limit='120GB', threads_per_worker=1)

In [3]:
client

Client Scheduler: tcp://127.0.0.1:36625 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 5 Cores: 5 Memory: 600.00 GB


In [4]:
client.restart()

Client Scheduler: tcp://127.0.0.1:36625 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 5 Cores: 5 Memory: 600.00 GB


In [5]:
region = 'cultivos_2019_simplify'
name_of_product = 's2_l2a_10m_scl_mexico'
begin = '2018-01-01'
end = '2019-12-31'
gwf_kwargs = {'region': region, 'product': name_of_product, 'begin': begin, 'end':end}
iterable = gwf_query(**gwf_kwargs)

In [6]:
lista_iter = list(iterable)

In [7]:
len(lista_iter)

53

In [8]:
date_today = datetime.today().strftime("%m-%d-%Y")

In [9]:
date_today='03-04-2020'

In [10]:
date_today

'03-04-2020'

In [11]:
mask_file='/LUSTRE/MADMEX/tasks/2020/1_clusterization_for_agriculture_labeling/madmex_masks_agricultura_pastizal/madmex_mascara_s2_2018_agricultura_pastizal_Veracruz_una_clase.tif'

In [12]:
crs_dict = {'proj': 'lcc',
 'lat_1': 17.5,
 'lat_2': 29.5,
 'lat_0': 12,
 'lon_0': -102,
 'x_0': 2500000,
 'y_0': 0,
 'datum': 'WGS84',
 'units': 'm',
 'no_defs': True,
 'a':6378137,
 'b':6378136.027241431,
 'wktext': True}
#+ellps=WGS84 +datum=WGS84

In [13]:
training_set = 'cultivos2019_inecol'

In [14]:
path = '/LUSTRE/MADMEX/tasks/2020/1_clusterization_for_agriculture_labeling/features_computed/all_dc_tiles'

In [15]:
if not os.path.exists(path):
    os.makedirs(path)

In [16]:
[(k,lista_iter[k][0]) for k in range(0,len(lista_iter))]

[(0, (47, -35)),
 (1, (47, -37)),
 (2, (48, -37)),
 (3, (47, -36)),
 (4, (48, -36)),
 (5, (40, -31)),
 (6, (39, -31)),
 (7, (38, -30)),
 (8, (39, -30)),
 (9, (40, -30)),
 (10, (39, -29)),
 (11, (38, -29)),
 (12, (40, -29)),
 (13, (38, -28)),
 (14, (40, -28)),
 (15, (38, -27)),
 (16, (39, -28)),
 (17, (39, -27)),
 (18, (38, -26)),
 (19, (39, -26)),
 (20, (40, -33)),
 (21, (40, -32)),
 (22, (39, -32)),
 (23, (37, -29)),
 (24, (40, -34)),
 (25, (41, -33)),
 (26, (42, -33)),
 (27, (41, -32)),
 (28, (42, -32)),
 (29, (41, -31)),
 (30, (42, -31)),
 (31, (43, -33)),
 (32, (44, -34)),
 (33, (43, -35)),
 (34, (45, -34)),
 (35, (44, -35)),
 (36, (45, -37)),
 (37, (43, -34)),
 (38, (44, -37)),
 (39, (43, -36)),
 (40, (45, -36)),
 (41, (45, -35)),
 (42, (44, -36)),
 (43, (42, -34)),
 (44, (41, -34)),
 (45, (37, -27)),
 (46, (37, -26)),
 (47, (37, -28)),
 (48, (41, -30)),
 (49, (46, -35)),
 (50, (46, -34)),
 (51, (46, -36)),
 (52, (46, -37))]

In [17]:
for var in lista_iter[23:24]:
    dc_tile = '%d_%d' % (var[0][0], var[0][1])
    control = 'processing_' + dc_tile + '.txt'
    file_control = open(path+control,'w+')
    file_control.write('Gridworkflow load\n')
    sr_0 = GridWorkflow.load(var[1],dask_chunks={'x': 1600, 'y': 1600,'time': 60})
    sr_0.attrs['geobox'] = var[1].geobox
    sr_0 = client.persist(sr_0)
    sr_0 = sr_0.apply(func=to_float, keep_attrs=True)
    sr_1 = sr_0.where(sr_0.pixel_qa.isin([2,4,5,6,7,8,11]))
    sr_1['ndvi'] = ((sr_1.nir - sr_1.red) / (sr_1.nir + sr_1.red)) * 10000
    sr_1['ndvi'].attrs['nodata'] = 0
    sr_1['gndvi'] = ((sr_1.nir - sr_1.green) / (sr_1.nir + sr_1.green)) * 10000
    sr_1['gndvi'].attrs['nodata'] = 0
    sr_1['avi'] = ((sr_1.nir*(1-sr_1.red))*(sr_1.nir-sr_1.red))**(1/3)
    sr_1['avi'].attrs['nodata'] = 0
    sr_1 = client.persist(sr_1)
    file_control.write('finished features computation\n')
    ndvi_mean_resampled = sr_1.ndvi.resample(time='M').mean()
    ndvi_mean_resampled = client.persist(ndvi_mean_resampled)
    ndvi_mean_resampled_drop_na = ndvi_mean_resampled.dropna('time',how='all')
    gndvi_mean_resampled = sr_1.gndvi.resample(time='M').mean()
    gndvi_mean_resampled = client.persist(gndvi_mean_resampled)
    gndvi_mean_resampled_drop_na = gndvi_mean_resampled.dropna('time',how='all')
    avi_mean_resampled = sr_1.avi.resample(time='M').mean()
    avi_mean_resampled = client.persist(avi_mean_resampled)
    avi_mean_resampled_drop_na = avi_mean_resampled.dropna('time',how='all')
    combined = xr.merge([ndvi_mean_resampled_drop_na,
                     gndvi_mean_resampled_drop_na,
                     avi_mean_resampled_drop_na])
    combined = client.persist(combined)
    features = list(combined.data_vars)
    s = '-'
    features_string = s.join(features)
    path_result = path + features_string + '/'
    path_result_dc_tile = os.path.join(path_result, date_today 
                                   + '/' + dc_tile + '/')
    if not os.path.exists(path_result_dc_tile):
        os.makedirs(path_result_dc_tile)
    combined.attrs = sr_0.attrs
    combined.coords['time'].attrs = sr_0.coords['time'].attrs
    combined.coords['x'].attrs = sr_0.coords['x'].attrs
    combined.coords['y'].attrs = sr_0.coords['y'].attrs
    file_control.write('beginning reading of segmentation\n')
    #read segmentation
    geom = GEOSGeometry(json.dumps(sr_0.geobox.geographic_extent.json))
    seg_name = 'seg_mex_s2_10m_scl_2018_2019'
    qs = PredictObject.objects.filter(the_geom__contained=geom,segmentation_information__name=seg_name)
    path_seg = qs[0].path
    with fiona.open(path_seg) as src:
        fc = list(src)
        crs = to_string(src.crs)
    fc_subset=((x['properties']['id'], x['geometry']) for x in fc)
    fc_sorted = sorted(fc_subset, key=itemgetter(0))
    iterable_zip = zip([k[1] for k in fc_sorted], [k[0] for k in fc_sorted])
    aff = Affine(*list(sr_0.affine)[0:6])
    dimensions_dataset = list(sr_0.coords)
    list_dimensions = [x for x in dimensions_dataset if x != 'time']
    lambda_function = lambda l_netcdf,l_test: l_netcdf[0] if l_netcdf[0] in l_test else l_netcdf[1]
    xdim = lambda_function(list_dimensions,['x','longitude'])
    ydim = lambda_function(list_dimensions,['y','latitude'])
    fc_raster = rasterize(iterable_zip, transform=aff,
                          out_shape=(sr_0.sizes[ydim], sr_0.sizes[xdim]),
                          dtype='float64', fill=np.nan)
    fc_raster = fc_raster.astype(int)
    fc_dataarray = xr.DataArray(fc_raster, dims=[ydim, xdim], name='features_id')
    file_control.write('beginning masking\n')
    geometry_extent = sr_0.geobox.extent.json
    geom_extent_schema={'properties':{'id':'int'},'geometry': 'Polygon'}
    layer_name = 'geom_extent_' + dc_tile
    geom_extent_file = path_result + '/' + date_today + '/' + dc_tile + '/' + layer_name + '.shp'
    with fiona.open(geom_extent_file,'w',
                driver='ESRI Shapefile',
                crs=crs_dict,
                layer=layer_name,
                schema=geom_extent_schema) as dst:
        dst.write({'properties':{'id':0},
                   'geometry': geometry_extent})
    mask_crop = gdal.Warp('', mask_file, format = 'VRT',
                          cutlineDSName=geom_extent_file,
                          cutlineLayer = layer_name,
                          cropToCutline=True)
    arr=mask_crop.ReadAsArray()
    combined_masked = combined.where(arr==1)
    file_control.write('finished masking\n')
    combined = xr.merge([fc_dataarray,combined_masked])
    combined = client.persist(combined)
    combined = combined.stack(z=(xdim, ydim))
    combined = combined.reset_index('z')
    combined = combined.drop([xdim,ydim])
    combined = combined.where((np.isfinite(combined['features_id'])), drop=True)
    combined = client.persist(combined)
    file_control.write('beginning dataframe construction\n')
    df = combined.to_dataframe()
    del sr_0
    del ndvi_mean_resampled_drop_na
    del gndvi_mean_resampled_drop_na
    del avi_mean_resampled_drop_na
    file_control.write('restarting client\n')
    client.restart()
    df=df.reset_index('time')
    df2 = df.groupby(['time','features_id'],as_index=False).agg({features[0]: 'median',
                                                                 features[1]: 'median',
                                                                 features[2]: 'median'})
    df2.index = df2['time']
    df2.drop(['time'], axis=1, inplace=True)
    times_df = [datetime.date(d).strftime('%Y-%m-%d') for d in df2.index.get_level_values('time').unique()]
    df3 = df2.loc[times_df[0]]
    df3.rename(columns={features[0]: features[0]+'_' + times_df[0],
                        features[1]: features[1]+'_' +times_df[0],
                        features[2]: features[2]+'_' +times_df[0]},
               inplace=True)
    df3=df3.reset_index('time')
    df3.drop(['time'],axis=1,inplace=True)
    for k in range(1,len(times_df)):
        df_subset = df2.loc[times_df[k]]
        df_subset.dropna(how='all', inplace=True)
        df_subset.rename(columns={features[0]: features[0]+'_' + times_df[k],
                                  features[1]: features[1]+'_' +times_df[k],
                                  features[2]: features[2]+'_' +times_df[k]},
                         inplace=True)
        df_subset=df_subset.reset_index('time')
        df_subset.drop(['time'],axis=1,inplace=True)
        df3 = df3.join(df_subset.set_index('features_id'), on='features_id') 
    file_control.write('finished building of df3\n')
    os.remove(geom_extent_file)
    os.remove(path_result + '/' + date_today + '/' + dc_tile + '/' + layer_name + '.dbf')
    os.remove(path_result + '/' + date_today + '/' + dc_tile + '/' + layer_name + '.shx')
    os.remove(path_result + '/' + date_today + '/' + dc_tile + '/' + layer_name + '.cpg')
    os.remove(path_result + '/' + date_today + '/' + dc_tile + '/' + layer_name + '.prj')
    file_control.write('beginning building of gdf\n')
    gdf = gpd.GeoDataFrame.from_features([feature for feature in fc], 
                                          crs = crs_dict)
    gdf = gdf.rename(columns={'id':'features_id'})
    gdf_df_join = gdf.merge(df3, on='features_id', how='left')
    gdf_df_join_reduced = gdf_df_join.drop(['features_id','geometry'],axis=1)
    gdf_df_join_reduced.dropna(how='all', inplace=True)
    t_keep_rows = gdf_df_join_reduced.index.values
    gdf_df_join_reduced2 = gdf_df_join.loc[gdf_df_join.index.isin(t_keep_rows)]
    file_control.write('beginning unsupervised classif\n')
    qs_t = TrainClassification.objects.filter(train_object__the_geom__contained=geom,
                                              training_set=training_set).prefetch_related('interpret_tag')
    l_tag_ids = []
    for element in qs_t:
        l_tag_ids.append(element.interpret_tag.id)
    nclusters = len(np.unique(l_tag_ids))
    kmeans = KMeans(n_clusters=nclusters, n_jobs=10)
    features_for_kmeans = [column for column in list(gdf_df_join_reduced2.columns) if column not in ['geometry','features_id']]
    kmeans.fit(gdf_df_join_reduced2[features_for_kmeans].fillna(0).values)
    model_filename = path_result_dc_tile + date_today + '_model_' +  dc_tile + '_nclusters_%d' % nclusters + '_aoi' + '.pkl'
    pickle.dump(kmeans, open(model_filename, "wb"))
    preds = kmeans.predict(gdf_df_join_reduced2[features_for_kmeans].fillna(0).values)
    gdf_df_clusters = gdf_df_join_reduced2.drop(features_for_kmeans,axis=1)
    gdf_df_clusters['preds'] = preds
    file_control.write('writing shapefile\n')
    clusters_filename = path_result_dc_tile + date_today + '_clusters_' +  dc_tile + '_nclusters_%d' % nclusters + '_aoi2' + '.shp'
    gdf_df_clusters.crs = crs_dict
    gdf_df_clusters.to_file(clusters_filename)
    file_control.write('finished processing\n')
    file_control.close()

OverflowError: cannot convert float infinity to integer